In [1]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor


df = pd.read_csv('Data/Dataset.csv', decimal =".", thousands=",")
df = df.drop(['Kota','Paslon 1','Paslon 2','Total'],axis=1)

scaler = MinMaxScaler()

df_minmax = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
dfX, dfY = df_minmax.iloc[:, :-1], df_minmax.iloc[:, [-1]]
kf = RepeatedKFold(n_splits=10, n_repeats = 3 , random_state=1)
kf.get_n_splits(dfX)
y = df['Partisipasi']


def flatten(x):
    return [item for sublist in x for item in sublist]


# Feature Selection
Feature selection merupakan sebuah teknik untuk memilih fitur yang paling relevan untuk membangun model. Fitur yang tidak relevan akan dihapus sehingga dapat mengurangi overfitting dan meningkatkan akurasi model.  
Terdapat 3 cara untuk melakukan feature selection, yaitu:
1. Filter Method
2. Wrapper Method
3. Embedded Method

Ketiga metode tersebut akan dipakai untuk mencari subset fitur terbaik dari dataset yang digunakan.

## Subset 

Dataset demografis dapat dibagi menjadi 4 subset besar yaitu:
1. Penduduk 
2. Pendidikan
3. Ekonomi
4. Sentimen

Kombinasi dari keempat subset tersebut akan diuji sebagai acuan dalam melakukan seleksi fitur.

Untuk mempermudah proses seleksi fitur, sebuah fungsi yang menggabungkan pembuatan model dan evaluasi model akan dibuat.  
Fungsi ini akan dibuaat di Utils.py

## Eval Models

In [3]:
def eval_models(dfX,y,kf):
  evals = []
  evals.append(evaluate_linreg(dfX,y,kf))  
  evals.append(evaluate_svr(dfX,y,kf))  
  evals.append(evaluate_dt(dfX,y,kf))  
  evals.append(evaluate_rf(dfX,y,kf))  
  evals.append(evaluate_xgboost(dfX,y,kf))  
  evals.append(evaluate_catboost(dfX,y,kf))  
  evals.append(evaluate_lightgbm(dfX,y,kf))  
  dfEval = pd.DataFrame(evals,columns=["Method","RSME","MAE","R2"])
  return dfEval

def compare_models(x1,x2,y,kf):
    dfEval_1 = eval_models(x1,y,kf)
    dfEval_2 = eval_models(x2,y,kf)
    dfDiff = dfEval_1.copy()
    dfDiff["RSME Diff"] = dfEval_1["RSME"] - dfEval_2["RSME"]
    dfDiff["MAE Diff"] = dfEval_1["MAE"] - dfEval_2["MAE"]
    dfDiff["R2 Diff"] = dfEval_1["R2"] - dfEval_2["R2"]
    dfDiff.drop(["Method","RSME","MAE","R2"],axis=1,inplace=True)
    return pd.concat([dfEval_1,dfEval_2,dfDiff],axis=1)


In [4]:
Subset = ["Penduduk","Pendidikan","Ekonomi","Sentimen"]
Fitur = ['Laju Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'Umur Squared','DPT'],['APM SD', 'APM SMP', 'APM SMA', 'Literate', 'TPAK', 'pendidikan_SD','pendidikan_SMP', 'pendidikan_SMA', 'pendidikan_Universitas', 'TK', 'SD', 'SMP', 'SMA', 'SMK', 'Universitas'],['Pengeluaran Bulanan','Persentase Kemiskinan', 'Indeks Pembangunan Manusia'],['Sentiment', 'Weighted Sentiment']

In [5]:
import itertools

combinations = []

for r in range(len(Fitur)+1):
    for combination in itertools.combinations(Fitur, r):
        combinations.append(combination)

combinations_name = []

for r in range(len(Subset)+1):
    for combination in itertools.combinations(Subset, r):
        combinations_name.append(combination)

dfResults = pd.DataFrame()


In [6]:
import Utils 

for i in range(1, 5):
  dfEval = dfX[Utils.flatten(list(combinations[i]))]
  results = Utils.eval_models(dfEval,y,kf)
  name = ' '.join(list(combinations_name[i]))
  results["Subset"] = name
  dfResults = pd.concat([dfResults,results])
  print(name)
  print(results)
dfResults

Penduduk
              Method      RSME       MAE        R2    Subset
0  Linear Regression  0.039146  0.031339  0.348727  Penduduk
1                SVR  0.045557  0.036148  0.117960  Penduduk
2      Decision Tree  0.056154  0.044861 -0.340104  Penduduk
3      Random Forest  0.042081  0.033502  0.247428  Penduduk
4            XGBoost  0.047816  0.037428  0.028327  Penduduk
5           Catboost  0.043899  0.035173  0.180984  Penduduk
6           LightGBM  0.041608  0.032873  0.264244  Penduduk
Pendidikan
              Method      RSME       MAE        R2      Subset
0  Linear Regression  0.046319  0.036545  0.088201  Pendidikan
1                SVR  0.047664  0.038493  0.034482  Pendidikan
2      Decision Tree  0.054108  0.043685 -0.244225  Pendidikan
3      Random Forest  0.040805  0.032964  0.292357  Pendidikan
4            XGBoost  0.044113  0.036136  0.173002  Pendidikan
5           Catboost  0.042237  0.034181  0.241829  Pendidikan
6           LightGBM  0.044534  0.035836  0.157129 

,Method,RSME,MAE,R2,Subset
0,Linear Regression,0.039146,0.031339,0.348727,Penduduk
1,SVR,0.045557,0.036148,0.117960,Penduduk
2,Decision Tree,0.056154,0.044861,-0.340104,Penduduk
3,Random Forest,0.042081,0.033502,0.247428,Penduduk
4,XGBoost,0.047816,0.037428,0.028327,Penduduk
5,Catboost,0.043899,0.035173,0.180984,Penduduk
6,LightGBM,0.041608,0.032873,0.264244,Penduduk
0,Linear Regression,0.046319,0.036545,0.088201,Pendidikan
1,SVR,0.047664,0.038493,0.034482,Pendidikan
2,Decision Tree,0.054108,0.043685,-0.244225,Pendidikan


Program Akan dirun hingga seluruh kombinasi subset fitur terpilih telah diuji.  
Hasil yang terbaik antar model antara lain:

## Hasil Evaluasi Model dengan Subset

| Method            | RSME      | MAE       | R2         | Subset                    |
|-------------------|-----------|-----------|------------|---------------------------|
| Linear Regression | 0,0391464 | 0,0313390 | 0,3487265  | Penduduk                  |
| SVR               | 0,0455569 | 0,0361479 | 0,1179601  | Penduduk                  |
| Decision Tree     | 0,0491608 | 0,0398620 | -0,0271097 | Penduduk Ekonomi Sentimen |
| Random Forest     | 0,0394041 | 0,0316941 | 0,3401233  | Penduduk Ekonomi Sentimen |
| XGBoost           | 0,0403263 | 0,0331756 | 0,3088773  | Penduduk Ekonomi Sentimen |
| LightGBM          | 0,0380277 | 0,0298752 | 0,3854200  | Penduduk Ekonomi Sentimen |
| Catboost          | 0,0384789 | 0,0308619 | 0,3707467  | Penduduk Ekonomi Sentimen |

## Filter
Filter merupakan metode yang menggunakan teknik statistik untuk memilih fitur yang paling relevan. Filter method tidak memerlukan model untuk memilih fitur namun biasanya menghasilkan akurasi yang lebih rendah dibandingkan dengan wrapper method dan embedded method. Oleh karena itu, filter akan dijadikan acuan untuk membandingkan akurasi dari kedua metode lainnya.

## SelectKBest Mutual Information

SelectKBest merupakan salah satu metode filter yang digunakan untuk memilih fitur yang paling relevan. SelectKBest menggunakan mutual_info_regression untuk menentukan fitur yang paling relevan.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression


def select_features(X_train, y_train, X_test,k):
 fs = SelectKBest(score_func=mutual_info_regression, k=k)
 fs.fit(X_train, y_train)
 mask = fs.get_support()
 new_features = dfX.columns[mask]
 return new_features
 


X_train, X_test, y_train, y_test = train_test_split(dfX, y, test_size=0.2, random_state=1)

dfResultsKBest = pd.DataFrame()
for i in range (1,5):
  features = select_features(X_train, y_train, X_test,i)
  dfEval = dfX[features]
  results = Utils.eval_models(dfEval,y,kf)
  name = ' ,'.join(features)
  results["Attributes"] = name
  dfResultsKBest = pd.concat([dfResultsKBest,results])
  print(name)
  print(results)
dfResultsKBest


Indeks Pembangunan Manusia
              Method      RSME       MAE        R2                  Attributes
0  Linear Regression  0.046515  0.036040  0.080456  Indeks Pembangunan Manusia
1                SVR  0.048040  0.039052  0.019208  Indeks Pembangunan Manusia
2      Decision Tree  0.059062  0.046238 -0.482498  Indeks Pembangunan Manusia
3      Random Forest  0.051971  0.041027 -0.147887  Indeks Pembangunan Manusia
4            XGBoost  0.057876  0.045275 -0.423546  Indeks Pembangunan Manusia
5           Catboost  0.050399  0.039891 -0.079501  Indeks Pembangunan Manusia
6           LightGBM  0.044722  0.034718  0.149976  Indeks Pembangunan Manusia
TK ,Indeks Pembangunan Manusia
              Method      RSME       MAE        R2   
0  Linear Regression  0.045945  0.035437  0.102886  \
1                SVR  0.047248  0.037882  0.051262   
2      Decision Tree  0.060734  0.046587 -0.567649   
3      Random Forest  0.048924  0.037962 -0.017241   
4            XGBoost  0.053222  0.041591

,Method,RSME,MAE,R2,Attributes
0,Linear Regression,0.046515,0.036040,0.080456,Indeks Pembangunan Manusia
1,SVR,0.048040,0.039052,0.019208,Indeks Pembangunan Manusia
2,Decision Tree,0.059062,0.046238,-0.482498,Indeks Pembangunan Manusia
3,Random Forest,0.051971,0.041027,-0.147887,Indeks Pembangunan Manusia
4,XGBoost,0.057876,0.045275,-0.423546,Indeks Pembangunan Manusia
5,Catboost,0.050399,0.039891,-0.079501,Indeks Pembangunan Manusia
6,LightGBM,0.044722,0.034718,0.149976,Indeks Pembangunan Manusia
0,Linear Regression,0.045945,0.035437,0.102886,"TK ,Indeks Pembangunan Manusia"
1,SVR,0.047248,0.037882,0.051262,"TK ,Indeks Pembangunan Manusia"
2,Decision Tree,0.060734,0.046587,-0.567649,"TK ,Indeks Pembangunan Manusia"


Hasil Hasil terbaik SelectKBest antara lain:

### Hasil Evaluasi Model dengan SelectKBest

| Method            | RSME      | MAE       | R2         | Fitur SelectKBest                                                                                                                                                                                                                                               |
|-------------------|-----------|-----------|------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Linear Regression | 0,0402219 | 0,0330630 |  0,3124502 | Laki, Perempuan, Umur, Umur Squared, Literate, pendidikan_SD, pendidikan_SMP, pendidikan_SMA, pendidikan_Universitas, TK, SD, SMP, SMK, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                             |
| SVR               | 0,0463914 | 0,0373972 |  0,0853518 | SD, Indeks Pembangunan Manusia                                                                                                                                                                                                                                  |
| Decision Tree     | 0,0509548 | 0,0409336 | -0,1034442 | Umur, Umur Squared, pendidikan_SD, pendidikan_SMA, SD, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                                                                                                              |
| Random Forest     | 0,0411838 | 0,0336680 |  0,2791694 | Umur, Umur Squared, pendidikan_SD, pendidikan_SMA, SD, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                                                                                                              |
| XGBoost           | 0,0396851 | 0,0330491 |  0,3306786 | Umur, Umur Squared, pendidikan_SD, pendidikan_SMA, SD, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                                                                                                              |
| LightGBM          | 0,0404047 | 0,0314712 |  0,3061873 | Laju Pertumbuhan, Laki, Perempuan, Umur, Umur Squared, APM SD, APM SMP, APM SMA, Literate, pendidikan_SD, pendidikan_SMP, pendidikan_SMA, pendidikan_Universitas, TK, SD, SMP, SMK, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT |
| Catboost          | 0,0393219 | 0,0324618 |  0,3428736 | Laju Pertumbuhan, Laki, Perempuan, Umur, Umur Squared, APM SD, APM SMA, Literate, pendidikan_SD, pendidikan_SMP, pendidikan_SMA, pendidikan_Universitas, TK, SD, SMP, SMK, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT          |

Seperti Hasil Modelling, kembali terlihat bahwa model linear regression, dan model ensemble learning memiliki akurasi yang lebih baik dibandingkan dengan model lainnya.

## ReliefF 

ReliefF merupakan salah satu metode filter yang digunakan untuk memilih fitur yang paling relevan. ReliefF menggunakan teknik pengurangan dimensi berbasis nearest neighbor untuk menentukan fitur yang paling relevan.  
Implementasi ReliefF yang digunakan berasal dari library skrebate.


In [3]:
from skrebate import ReliefF

fs = ReliefF(n_features_to_select=4, n_neighbors=100, n_jobs=-1)
fs.fit(dfX.values, y)
sorted_dict = {}
for name, score in zip(dfX.columns, fs.feature_importances_):
    sorted_dict[name] = score
sorted_dict = dict(sorted(sorted_dict.items(), key=lambda item: item[1], reverse=True))
sorted_dict

{'Indeks Pembangunan Manusia': 0.13284091911102613,
 'SD': 0.12034276029063282,
 'pendidikan_SMA': 0.11226924394021717,
 'pendidikan_SD': 0.09862159114928643,
 'DPT': 0.08951284040805399,
 'Persentase Kemiskinan': 0.08670741003840189,
 'pendidikan_Universitas': 0.0770181753771101,
 'Pengeluaran Bulanan': 0.06752418168824369,
 'Literate': 0.058178733750543235,
 'Perempuan': 0.052779796704443833,
 'Laki': 0.04879084066123379,
 'TK': 0.04016274053710977,
 'TPAK': 0.03947882612619671,
 'Laju Pertumbuhan': 0.03707727177524649,
 'Umur': 0.03485968154348226,
 'Umur Squared': 0.03442564055727986,
 'SMK': 0.033480552990718025,
 'SMP': 0.02936339912835733,
 'Sentiment': 0.017851810718429697,
 'Weighted Sentiment': 0.017851810718429697,
 'APM SD': 0.015954416625980503,
 'SMA': 0.014888048240001402,
 'pendidikan_SMP': 0.013984129438205239,
 'APM SMP': 0.013054998632209899,
 'APM SMA': 0.012096499861815853,
 'Universitas': -0.013497161469518246}

Hasil diatas merupakan feature importance yang dihasilkan oleh ReliefF. Hasil tersebut akan diuji menggunakan semua model untuk melihat akurasi yang dihasilkan.

## Hasil Evaluasi Model dengan ReliefF
| Method           | RSME     | MAE      | R2        | Fitur SelectKBest                                                                                                                                                                                                                                                                 |
|------------------|----------|----------|-----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| LinearRegression | 0,040827 | 0,032958 |  0,291616 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan Literate Perempuan Laki TK TPAK Laju Pertumbuhan Umur Umur Squared SMK SMP Sentiment Weighted Sentiment APM SD SMA pendidikan_SMP APM SMP APM SMA |
| SVR              | 0,046285 | 0,037046 |  0,089537 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD                                                                                                                                                                                                                        |
| DecisionTree     | 0,052342 | 0,041820 | -0,164340 | Indeks Pembangunan Manusia SD                                                                                                                                                                                                                                                     |
| RandomForest     | 0,040009 | 0,032160 |  0,319714 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan Literate Perempuan Laki TK TPAK Laju Pertumbuhan Umur Umur Squared SMK SMP Sentiment Weighted Sentiment APM SD SMA pendidikan_SMP APM SMP APM SMA |
| XGBoost          | 0,042302 | 0,035010 |  0,239495 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan                                                                                                                                                   |
| LightGBM         | 0,042743 | 0,034286 |  0,223575 | Indeks Pembangunan Manusia SD                                                                                                                                                                                                                                                     |
| Catboost         | 0,039739 | 0,032746 |  0,328858 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan Literate Perempuan Laki TK TPAK Laju Pertumbuhan Umur Umur Squared SMK SMP Sentiment Weighted Sentiment APM SD SMA pendidikan_SMP APM SMP APM SMA |

Hasil reliefF jauh lebih buruk dibandingkan Hasil yang diberikan subset. 

## Wrapper Method

Seperti namanya, metode wrapper membalut model machine learning dan mengukur kinerja model berdasarkan fitur yang dipilih. Metode wrapper memilih fitur berdasarkan kinerja model yang dibalut.  
Metode wrapper identik dengan penggunaan metode optimisasi sebagai algoritma pencari fitur. Beberapa metode optimisasi yang digunakan antara lain:
1. Genetic Algorithm
2. Particle Swarm Optimization
3. Dragonfly Algorithm
4. Grey Wolf Optimizer
5. Harris Hawks Optimization


## Genetic Algorithm 1

GA merupakan teknik seleksi fitur yang menggunakan konsep algoritma genetika. Implementasi dari GA yang digunakan berasal dari library genetic_selection.  
Estimator yang dipakai adalah Linear Regression, dan Random Forest.  
Kedua estimator tersebut dipakai karena Linear regression memiliki hasil terbaik diantara model yang tidak berbasis tree dan random forest merupakan model yang seimbang dalam hal akurasi dan kecepatan.

In [3]:
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
import math


def RMSEScore(actual, predict):
  score = math.sqrt(mean_squared_error(actual,predict))
  return score

estimator = LinearRegression()

report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 

# range diganti untuk mempercepat training(visualisasi), range(1,15) untuk full range
for i in range (14,15):
  model = GeneticSelectionCV(estimator,
                                cv = kf,
                                verbose = 0,
                                scoring = RMSEScore,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  model = model.fit(dfX, y)
  genfeats = dfX.columns[model.support_]
  print('Features:', genfeats)

  cv_score = model.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)
  
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore

Features: Index(['APM SD', 'pendidikan_SMA', 'Universitas',
       'Indeks Pembangunan Manusia'],
      dtype='object')


### Genetic Algorithm 1 dengan basis Linear Regression

Angka paling dekat ke 0 merupakan hasil terbaik.

| Maximum Feats | No of Feats | Scores         | Chosen Feats                                                                                                                           |
|---------------|-------------|----------------|----------------------------------------------------------------------------------------------------------------------------------------|
|            11 |          10 | -0,03525054205 | Laki', 'Umur', 'Umur Squared', 'APM SMA', 'SMA', 'SMK','Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT','Weighted Sentiment' |
|             6 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|             7 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|             8 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|             9 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|            10 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|            12 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|            13 |           6 |  -0,0357277118 | Laki', 'Umur', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                      |
|             4 |           5 | -0,03578968066 | Laki', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                              |
|             5 |           5 | -0,03578968066 | Laki', 'Umur Squared', 'TK', 'SMA', 'DPT'                                                                                              |
|             3 |           4 | -0,03620506409 | Laki', 'Umur Squared', 'SMA', 'DPT'                                                                                                    |
|             2 |           3 | -0,03648223628 | Laki', 'SMA', 'DPT'                                                                                                                    |
|             1 |           2 | -0,03901257704 | Perempuan', 'DPT'                                                                                                                      |
|             0 |           1 | -0,04497598781 | Indeks Pembangunan Manusia'                                                                                                            |

Model lalu dibandingkan dengan menggunakan fitur yang dipilih.

### Hasil Evaluasi Model dengan Genetic Algorithm 1 dengan basis Linear Regression

| index | Method            | RSME          | MAE           | R2            |
|-------|-------------------|---------------|---------------|---------------|
| 0     | Linear Regression | 0,03557438313 | 0,02960915453 | 0,4621587294  |
| 5     | Catboost          | 0,03595601986 | 0,0294830146  | 0,4505570674  |
| 6     | LightGBM          | 0,03745373452 | 0,02988114094 | 0,4038306784  |
| 3     | Random Forest     | 0,03829253303 | 0,0310056919  | 0,3768285337  |
| 4     | XGBoost           | 0,03860404978 | 0,03184476503 | 0,3666480642  |
| 1     | SVR               | 0,0480342043  | 0,03912312819 | 0,01942555658 |
| 2     | Decision Tree     | 0,0512191436  | 0,04058513843 | -0,1149207382 |

In [2]:
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import *
from sklearn.ensemble import RandomForestRegressor
import math


def RMSEScore(actual, predict):
  score = math.sqrt(mean_squared_error(actual,predict))
  return score

estimator = RandomForestRegressor(n_estimators=100, random_state=0)

report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 

# range diganti untuk mempercepat training(visualisasi), range(1,15) untuk full range
for i in range (2,3):
  model = GeneticSelectionCV(estimator,
                                cv = kf,
                                verbose = 0,
                                scoring = RMSEScore,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  model = model.fit(dfX, y)
  genfeats = dfX.columns[model.support_]
  print('Features:', genfeats)

  cv_score = model.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)
  
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore

Features: Index(['SMK'], dtype='object')


### Genetic Algorithm 1 menggunakan basis Random Forest

| Maximum Feats | No of Feats | Scores    | Chosen Feats                                                                                                                              |
|---------------|-------------|-----------|-------------------------------------------------------------------------------------------------------------------------------------------|
| 8             | 8           | -0,036165 | Laki', 'Umur', 'Umur Squared', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'               |
| 10            | 8           | -0,036165 | Laki', 'Umur', 'Umur Squared', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'               |
| 13            | 8           | -0,036165 | Laki', 'Umur', 'Umur Squared', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'               |
| 7             | 8           | -0,036322 | Laki', 'Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Weighted Sentiment'                 |
| 9             | 8           | -0,036483 | Laki', 'Umur', 'APM SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'                   |
| 11            | 10          | -0,036618 | Laki', 'Perempuan', 'Umur', 'Umur Squared', 'SMP', 'SMA','Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Weighted Sentiment' |
| 6             | 7           | -0,036674 | Perempuan', 'Umur', 'Umur Squared', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Weighted Sentiment'                       |
| 12            | 8           | -0,036674 | Laju Pertumbuhan', 'Umur', 'Umur Squared', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'   |
| 4             | 5           | -0,036801 | APM SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT','Weighted Sentiment'                                                 |
| 5             | 6           | -0,036845 | Umur', 'Umur Squared', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Weighted Sentiment'                                    |
| 3             | 4           | -0,03774  | Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT','Weighted Sentiment'                                                            |
| 2             | 3           | -0,038195 | Pengeluaran Bulanan', 'Indeks Pembangunan Manusia','Weighted Sentiment'                                                                   |
| 1             | 2           | -0,039841 | Pengeluaran Bulanan', 'Indeks Pembangunan Manusia'                                                                                        |
| 0             | 1           | -0,044483 | Pengeluaran Bulanan'                                                                                                                      |

### Hasil Evaluasi Model dengan Genetic Algorithm 1 dengan basis Random Forest

| Index | Method            | RSME    | MAE     | R2       |
|-------|-------------------|---------|---------|----------|
| 5     | Catboost          | 0,03688 | 0,02968 | 0,42195  |
| 3     | Random Forest     | 0,03717 | 0,03020 | 0,41300  |
| 6     | LightGBM          | 0,03786 | 0,02964 | 0,39081  |
| 4     | XGBoost           | 0,03813 | 0,03140 | 0,38227  |
| 0     | Linear Regression | 0,03870 | 0,03088 | 0,36363  |
| 1     | SVR               | 0,04786 | 0,03918 | 0,02665  |
| 2     | Decision Tree     | 0,04977 | 0,03914 | -0,05278 |

## Swarm Algorithm dan Bio-Inspired Algorithm

Semua algorithm akan dirun dengan 3 kondisi dengan model random forest untuk membandingkan hasil antar kondisi.
1. Konfigurasi 1 =  Iterasi = 10, Populasi = 30
2. Konfigurasi 2 =  Iterasi = 30, Populasi = 30
3. Konfigurasi 3 =  Iterasi = 100, Populasi = 30

Penambahan populasi tidak dilakukan karena terlalu besar komputasi yang dibutuhkan.

## Particle Swarm Optimization

Particle Swarm Optimization atau PSO merupakan teknik optimasi berbasis swarm optimization. Varian Binary dari PSO sering digunakan dalam seleksi fitur.


In [7]:
import math
from sklearn.metrics import mean_squared_error
from zoofs import ParticleSwarmOptimization

def objective_function_topass(model, X_train, y_train, X_valid, y_valid):
  # Objective function merupakan fungsi yang akan dioptimasi hasilnya
  # RMSE merupakan nilai yang akan dioptimasi
  preds = []
  vals = []
  for train_index, test_index in kf.split(X_train):
    xtrain, xtest = X_train.iloc[train_index], X_valid.iloc[test_index]
    ytrain, ytest = y_train.iloc[train_index], y_valid.iloc[test_index]
    model.fit(xtrain, ytrain)
    preds.append(model.predict(xtest))
    vals.append(ytest)

  score = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))

  return score



# iteration diganti untuk mempercepat training(visualisasi), gunakan 10,30,100 untuk full range
algo_object=ParticleSwarmOptimization(objective_function_topass,n_iteration=10,population_size=2,minimize=True)
RF_Model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1, n_jobs=-1)

algo_object.fit(RF_Model,dfX, y, dfX, y,verbose=True)
print(algo_object.best_feature_list)
algo_object.plot_history()

 [ 2023-06-06 16:42:30,033 ] Finished iteration #0 with objective value 0.03993526006320904. Current best value is 0.03993526006320904 
 [ 2023-06-06 16:42:39,233 ] Finished iteration #1 with objective value 0.04047258485011633. Current best value is 0.03993526006320904 
 [ 2023-06-06 16:42:48,457 ] Finished iteration #2 with objective value 0.039051104144212534. Current best value is 0.039051104144212534 
 [ 2023-06-06 16:42:57,590 ] Finished iteration #3 with objective value 0.0396776854538726. Current best value is 0.039051104144212534 
 [ 2023-06-06 16:43:07,030 ] Finished iteration #4 with objective value 0.039597165988187286. Current best value is 0.039051104144212534 
 [ 2023-06-06 16:43:16,246 ] Finished iteration #5 with objective value 0.03921744094642794. Current best value is 0.039051104144212534 
 [ 2023-06-06 16:43:25,630 ] Finished iteration #6 with objective value 0.03942683782395794. Current best value is 0.039051104144212534 
 [ 2023-06-06 16:43:34,870 ] Finished iter

['Umur', 'APM SMP', 'APM SMA', 'Literate', 'pendidikan_SD', 'pendidikan_SMA', 'SD', 'SMP', 'Pengeluaran Bulanan', 'Persentase Kemiskinan', 'DPT', 'Sentiment', 'Weighted Sentiment']


### Hasil PSO dengan basis Random Forest

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                              |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| PSO                 | 0,03722           | 8                         | 10               | 30              | 912,4    | 9          | ['Umur',   'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan   Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']             |
| PSO                 | 0,03733           | 24                        | 30               | 30              | 2.769,7  | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMA', 'SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'] |
| PSO                 | 0,03714           | 21                        | 100              | 30              | 9.575,5  | 9          | ['Laki',   'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan   Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']             |

## Dragonfly Algorithm

DFA atau Dragonfly Algorithm merupakan teknik optimasi berbasis bio-inspired algorithm. Varian Binary dari DFA sering digunakan dalam seleksi fitur.

Dragonfly Algorithm memiliki 4 method yang dapat diubah, yaitu:
1. Linear
2. Random
3. Quadraic 
4. Sinusoidal

[Berdasarkan paper yang dikutip zoofs](https://doi.org/10.1016/j.neucom.2015.06.083), sinusoidal merupakan method yang menghasilkan hasil terbaik. Oleh karena itu, method yang digunakan adalah sinusoidal.

In [ ]:
from zoofs import DragonFlyOptimization

def objective_function_topass(model, X_train, y_train, X_valid, y_valid):
  preds = []
  vals = []
  for train_index, test_index in kf.split(X_train):
    xtrain, xtest = X_train.iloc[train_index], X_valid.iloc[test_index]
    ytrain, ytest = y_train.iloc[train_index], y_valid.iloc[test_index]
    model.fit(xtrain, ytrain)
    preds.append(model.predict(xtest))
    vals.append(ytest)

  score = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))

  return score



algo_object=DragonFlyOptimization(objective_function_topass,n_iteration=10,population_size=30,minimize=True)
RF_Model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)    
                                
algo_object.fit(RF_Model,dfX, y, dfX, y,verbose=True)
print(algo_object.best_feature_list)
algo_object.plot_history()

### Hasil Dragonfly Algorithm dengan basis Random Forest

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                        |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| DFA                 | 0,03806           | 13                        | 10               | 30              | 1.074,4  | 13         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SMP',   'pendidikan_SD', 'pendidikan_SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'] |
| DFA                 | 0,03804           | 25                        | 30               | 30              | 3.447,8  | 11         | ['Perempuan',   'Umur', 'Umur Squared', 'pendidikan_SMP', 'SMP', 'SMA', 'Pengeluaran   Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT',   'Weighted Sentiment']                              |
| DFA                 | 0,03769           | 59                        | 100              | 30              | 11.621,0 | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMP', 'pendidikan_SMP', 'SMP', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment']                                              |

### Grey Wolf Optimizer

Grey Wolf Optimizer atau GWO merupakan teknik optimasi berbasis bio-inspired algorithm. Varian Binary dari GWO sering digunakan dalam seleksi fitur.  
GWO terinspirasi dari perilaku serigala dalam mencari mangsa. Serigala akan berburu dengan cara berkelompok dan berkoordinasi untuk mengejar mangsa. Serigala yang paling dominan akan menjadi alpha, yang kedua akan menjadi beta, dan yang ketiga akan menjadi delta. Serigala yang lain akan menjadi omega. Posisi alpha, beta, dan delta akan diupdate setiap iterasi.  

GWO memiliki parameter method, method merupakan parameter yang digunakan untuk mengupdate posisi alpha, beta, dan delta. Terdapat 2 method yang dapat digunakan.  
[Pada paper yang dikutip oleh Zoofs]((https://doi.org/10.1016/j.neucom.2015.06.083)), Metode kedua menghasilkan hasil yang lebih baik dibandingkan metode pertama. Oleh karena itu, metode 2 yang digunakan dalam eksperimen ini. 

In [ ]:
from zoofs import GreyWolfOptimization

def objective_function_topass(model, X_train, y_train, X_valid, y_valid):
  preds = []
  vals = []
  for train_index, test_index in kf.split(X_train):
    xtrain, xtest = X_train.iloc[train_index], X_valid.iloc[test_index]
    ytrain, ytest = y_train.iloc[train_index], y_valid.iloc[test_index]
    model.fit(xtrain, ytrain)
    preds.append(model.predict(xtest))
    vals.append(ytest)

  score = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))

  return score



algo_object=GreyWolfOptimization(objective_function_topass,n_iteration=10,population_size=30,minimize=True, method=2)
RF_Model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)                                    
algo_object.fit(RF_Model,dfX, y, dfX, y,verbose=True)
print(algo_object.best_feature_list)
algo_object.plot_history()

### Hasil Grey Wolf Optimizer dengan basis Random Forest

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                        |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| DFA                 | 0,03806           | 13                        | 10               | 30              | 1.074,4  | 13         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SMP',   'pendidikan_SD', 'pendidikan_SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'] |
| DFA                 | 0,03804           | 25                        | 30               | 30              | 3.447,8  | 11         | ['Perempuan',   'Umur', 'Umur Squared', 'pendidikan_SMP', 'SMP', 'SMA', 'Pengeluaran   Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT',   'Weighted Sentiment']                              |
| DFA                 | 0,03769           | 59                        | 100              | 30              | 11.621,0 | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMP', 'pendidikan_SMP', 'SMP', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment']                                              |

### Harris Hawks Optimization

Harris Hawks Optimization atau HHO merupakan teknik optimasi berbasis bio-inspired algorithm. Varian Binary dari HHO sering digunakan dalam seleksi fitur. HHO terinspirasi dari cara elang berburu, HHO membedakan dirinya dengan teknik seleksi fitur metaheuristic lainya dengan menggunakan dua fase eksplorasai dan empat fase eksploitasi. 

In [ ]:
from zoofs import HarrisHawkOptimization

def objective_function_topass(model, X_train, y_train, X_valid, y_valid):
  preds = []
  vals = []
  for train_index, test_index in kf.split(X_train):
    xtrain, xtest = X_train.iloc[train_index], X_valid.iloc[test_index]
    ytrain, ytest = y_train.iloc[train_index], y_valid.iloc[test_index]
    model.fit(xtrain, ytrain)
    preds.append(model.predict(xtest))
    vals.append(ytest)

  score = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))

  return score


algo_object=HarrisHawkOptimization(objective_function_topass,n_iteration=10,population_size=30,minimize=True)

RF_Model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1, n_jobs=-1)                                      
algo_object.fit(RF_Model,dfX, y, dfX, y,verbose=True)
print(algo_object.best_feature_list)
algo_object.plot_history()

### Hasil Harris Hawk Optimization dengan basis Random Forest

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                                                                                                                           |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| HHO                 | 0,03802           | 0                         | 10               | 30              | 1.665,0  | 21         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SD', 'APM SMP', 'APM   SMA', 'Literate', 'pendidikan_SD', 'pendidikan_SMA',   'pendidikan_Universitas', 'TK', 'SD', 'SMP', 'SMK', 'Universitas',   'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment',   'Weighted Sentiment'] |
| HHO                 | 0,03814           | 28                        | 30               | 30              | 6.223,7  | 17         | ['Laju   Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'APM SMP', 'APM SMA',   'pendidikan_SD', 'pendidikan_SMP', 'TK', 'SD', 'SMA', 'SMK', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment']                                                                            |
| HHO                 | 0,03823           | 10                        | 100              | 30              | 19.950,2 | 18         | ['Perempuan',   'Umur', 'APM SD', 'APM SMA', 'TPAK', 'pendidikan_SD', 'pendidikan_SMA',   'pendidikan_Universitas', 'SD', 'SMP', 'SMA', 'SMK', 'Universitas',   'Pengeluaran Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia',   'DPT', 'Weighted Sentiment']                                           |

### Genetic Algorithm ZooFS 

ZooFS juga memiliki varian GA. Karena GA sudah diimplementasi dengan package lain, GA ZooFS akan dibandingkan dengan GA diatas.


In [ ]:
def objective_function_topass(model, X_train, y_train, X_valid, y_valid):
  preds = []
  vals = []
  for train_index, test_index in kf.split(X_train):
    xtrain, xtest = X_train.iloc[train_index], X_valid.iloc[test_index]
    ytrain, ytest = y_train.iloc[train_index], y_valid.iloc[test_index]
    model.fit(xtrain, ytrain)
    preds.append(model.predict(xtest))
    vals.append(ytest)

  score = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))

  return score

from zoofs import GeneticOptimization
algo_object=GeneticOptimization(objective_function_topass,n_iteration=10,population_size=30,minimize=True)

RF_Model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1, n_jobs=-1)                                      
algo_object.fit(RF_Model,dfX, y, dfX, y,verbose=True)
print(algo_object.best_feature_list)
algo_object.plot_history()

### Hasil Genetic Algorithm Zoofs dengan basis Random Forest

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                              |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| GA                  | 0,03888           | 8                         | 10               | 30              | 666,7    | 12         | ['Laki',   'Perempuan', 'Umur', 'Umur Squared', 'pendidikan_SMA',   'pendidikan_Universitas', 'TK', 'SD', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Weighted Sentiment']                             |
| GA                  | 0,03830           | 9                         | 30               | 30              | 345,4    | 9          | ['Laki',   'Umur Squared', 'APM SMP', 'pendidikan_SD', 'TK', 'SD', 'SMP', 'Pengeluaran   Bulanan', 'Weighted Sentiment']                                                                                                  |
| GA                  | 0,03794           | 10                        | 100              | 30              | 529,3    | 13         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur Squared', 'APM SMA', 'pendidikan_SD',   'pendidikan_SMP', 'SD', 'Universitas', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment'] |

## Rekap Hasil Seleksi Fitur Per Skenario

### 10 Iterasi

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                                                                                                                           |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| GA                  | 0,03888           | 8                         | 10               | 30              | 666,7    | 12         | ['Laki',   'Perempuan', 'Umur', 'Umur Squared', 'pendidikan_SMA',   'pendidikan_Universitas', 'TK', 'SD', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Weighted Sentiment']                                                                                                                          |
| GWO                 | 0,03866           | 0                         | 10               | 30              | 867,3    | 10         | ['Umur',   'Umur Squared', 'APM SMP', 'pendidikan_SD', 'SD', 'SMP', 'SMA', 'Indeks   Pembangunan Manusia', 'DPT', 'Weighted Sentiment']                                                                                                                                                                                |
| PSO                 | 0,03722           | 8                         | 10               | 30              | 912,4    | 9          | ['Umur',   'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan   Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                                                                                                          |
| DFA                 | 0,03806           | 13                        | 10               | 30              | 1.074,4  | 13         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SMP',   'pendidikan_SD', 'pendidikan_SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                                    |
| HHO                 | 0,03802           | 0                         | 10               | 30              | 1.665,0  | 21         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SD', 'APM SMP', 'APM   SMA', 'Literate', 'pendidikan_SD', 'pendidikan_SMA',   'pendidikan_Universitas', 'TK', 'SD', 'SMP', 'SMK', 'Universitas',   'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment',   'Weighted Sentiment'] |

### 30 Iterasi
| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                                                |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| GA                  | 0,03830           | 9                         | 30               | 30              | 345,4    | 9          | ['Laki',   'Umur Squared', 'APM SMP', 'pendidikan_SD', 'TK', 'SD', 'SMP', 'Pengeluaran   Bulanan', 'Weighted Sentiment']                                                                                                                    |
| PSO                 | 0,03733           | 24                        | 30               | 30              | 2.769,7  | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMA', 'SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                   |
| DFA                 | 0,03804           | 25                        | 30               | 30              | 3.447,8  | 11         | ['Perempuan',   'Umur', 'Umur Squared', 'pendidikan_SMP', 'SMP', 'SMA', 'Pengeluaran   Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT',   'Weighted Sentiment']                                                      |
| GWO                 | 0,03835           | 13                        | 30               | 30              | 4.046,6  | 11         | ['Laju   Pertumbuhan', 'Laki', 'Umur', 'Umur Squared', 'SMA', 'Universitas',   'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment',   'Weighted Sentiment']                                                             |
| HHO                 | 0,03814           | 28                        | 30               | 30              | 6.223,7  | 17         | ['Laju   Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'APM SMP', 'APM SMA',   'pendidikan_SD', 'pendidikan_SMP', 'TK', 'SD', 'SMA', 'SMK', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment'] |

### 100 Iterasi
| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                                                |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| GA                  | 0,03830           | 9                         | 30               | 30              | 345,4    | 9          | ['Laki',   'Umur Squared', 'APM SMP', 'pendidikan_SD', 'TK', 'SD', 'SMP', 'Pengeluaran   Bulanan', 'Weighted Sentiment']                                                                                                                    |
| PSO                 | 0,03733           | 24                        | 30               | 30              | 2.769,7  | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMA', 'SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                   |
| DFA                 | 0,03804           | 25                        | 30               | 30              | 3.447,8  | 11         | ['Perempuan',   'Umur', 'Umur Squared', 'pendidikan_SMP', 'SMP', 'SMA', 'Pengeluaran   Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT',   'Weighted Sentiment']                                                      |
| GWO                 | 0,03835           | 13                        | 30               | 30              | 4.046,6  | 11         | ['Laju   Pertumbuhan', 'Laki', 'Umur', 'Umur Squared', 'SMA', 'Universitas',   'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment',   'Weighted Sentiment']                                                             |
| HHO                 | 0,03814           | 28                        | 30               | 30              | 6.223,7  | 17         | ['Laju   Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'APM SMP', 'APM SMA',   'pendidikan_SD', 'pendidikan_SMP', 'TK', 'SD', 'SMA', 'SMK', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment'] |


## Rekap Hasil Secara Keseluruhan

| Optimization Method | Best Value (RMSE) | Best Feature at iteration | Total Iterations | Population Size | Time (s) | N Features | Feature List                                                                                                                                                                                                                                                                                                           |
|---------------------|-------------------|---------------------------|------------------|-----------------|----------|------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| PSO                 | 0,03714           | 21                        | 100              | 30              | 9.575,5  | 9          | ['Laki',   'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan   Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                                                                                                          |
| PSO                 | 0,03722           | 8                         | 10               | 30              | 912,4    | 9          | ['Umur',   'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan   Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                                                                                                          |
| PSO                 | 0,03733           | 24                        | 30               | 30              | 2.769,7  | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMA', 'SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                                                                                              |
| DFA                 | 0,03769           | 59                        | 100              | 30              | 11.621,0 | 11         | ['Laki',   'Perempuan', 'Umur', 'APM SMP', 'pendidikan_SMP', 'SMP', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment']                                                                                                                                                 |
| GA                  | 0,03794           | 10                        | 100              | 30              | 529,3    | 13         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur Squared', 'APM SMA', 'pendidikan_SD',   'pendidikan_SMP', 'SD', 'Universitas', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                              |
| HHO                 | 0,03802           | 0                         | 10               | 30              | 1.665,0  | 21         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SD', 'APM SMP', 'APM   SMA', 'Literate', 'pendidikan_SD', 'pendidikan_SMA',   'pendidikan_Universitas', 'TK', 'SD', 'SMP', 'SMK', 'Universitas',   'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment',   'Weighted Sentiment'] |
| DFA                 | 0,03804           | 25                        | 30               | 30              | 3.447,8  | 11         | ['Perempuan',   'Umur', 'Umur Squared', 'pendidikan_SMP', 'SMP', 'SMA', 'Pengeluaran   Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia', 'DPT',   'Weighted Sentiment']                                                                                                                                 |
| DFA                 | 0,03806           | 13                        | 10               | 30              | 1.074,4  | 13         | ['Laju   Pertumbuhan', 'Perempuan', 'Umur', 'Umur Squared', 'APM SMP',   'pendidikan_SD', 'pendidikan_SMP', 'SMA', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']                                                                                                    |
| GWO                 | 0,03812           | 76                        | 100              | 30              | 13.173,6 | 14         | ['Laju   Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'Umur Squared', 'APM SMP', 'APM   SMA', 'pendidikan_SD', 'pendidikan_SMP', 'Universitas', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Weighted Sentiment']                                                                                      |
| HHO                 | 0,03814           | 28                        | 30               | 30              | 6.223,7  | 17         | ['Laju   Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'APM SMP', 'APM SMA',   'pendidikan_SD', 'pendidikan_SMP', 'TK', 'SD', 'SMA', 'SMK', 'Pengeluaran   Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted   Sentiment']                                                                            |
| HHO                 | 0,03823           | 10                        | 100              | 30              | 19.950,2 | 18         | ['Perempuan',   'Umur', 'APM SD', 'APM SMA', 'TPAK', 'pendidikan_SD', 'pendidikan_SMA',   'pendidikan_Universitas', 'SD', 'SMP', 'SMA', 'SMK', 'Universitas',   'Pengeluaran Bulanan', 'Persentase Kemiskinan', 'Indeks Pembangunan Manusia',   'DPT', 'Weighted Sentiment']                                           |
| GA                  | 0,03830           | 9                         | 30               | 30              | 345,4    | 9          | ['Laki',   'Umur Squared', 'APM SMP', 'pendidikan_SD', 'TK', 'SD', 'SMP', 'Pengeluaran   Bulanan', 'Weighted Sentiment']                                                                                                                                                                                               |
| GWO                 | 0,03835           | 13                        | 30               | 30              | 4.046,6  | 11         | ['Laju   Pertumbuhan', 'Laki', 'Umur', 'Umur Squared', 'SMA', 'Universitas',   'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment',   'Weighted Sentiment']                                                                                                                                        |
| GWO                 | 0,03866           | 0                         | 10               | 30              | 867,3    | 10         | ['Umur',   'Umur Squared', 'APM SMP', 'pendidikan_SD', 'SD', 'SMP', 'SMA', 'Indeks   Pembangunan Manusia', 'DPT', 'Weighted Sentiment']                                                                                                                                                                                |
| GA                  | 0,03888           | 8                         | 10               | 30              | 666,7    | 12         | ['Laki',   'Perempuan', 'Umur', 'Umur Squared', 'pendidikan_SMA',   'pendidikan_Universitas', 'TK', 'SD', 'Pengeluaran Bulanan', 'Indeks   Pembangunan Manusia', 'DPT', 'Weighted Sentiment']                                                                                                                          |

## Embedded Feature Selection

Embedded feature selection merupakan teknik seleksi fitur yang menggabungkan proses pelatihan model dan seleksi fitur. Teknik ini memanfaatkan algoritma pembelajaran mesin untuk mengekstraksi fitur yang paling penting.  

## CatBoost Recursive Feature Elimination

CatBoost memiliki algoritma feature selection yang disebut Recursive Feature Elimination (RFE). Algoritma ini menghilangkan fitur yang memiliki pengaruh paling kecil terhadap performa model. Algoritma ini menghilangkan fitur satu per satu hingga performa model menurun. RFE pada CatBoost merupakan algoritma feature selection embedded karena proses pelatihan model dan seleksi fitur dilakukan secara bersamaan.  


In [2]:
from catboost import CatBoostRegressor


regressor = CatBoostRegressor(eval_metric='RMSE', 
                              iterations = 100,
                              random_state = 1
                              )

X_train, X_test, y_train, y_test = train_test_split(dfX, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 

rfe_dict = regressor.select_features(X = X_train, 
                                     y = y_train, 
                                     eval_set = (X_val,y_val), 
                                     features_for_select = '0-25', 
                                     num_features_to_select = 10, 
                                     steps = 5, 
                                     verbose = False,
                                     train_final_model = False, 
                                     plot = True 
                                     )


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.134398
Step #1 out of 5

bestTest = 0.0453662099
bestIteration = 64

Shrink model to first 65 iterations.
Feature #12 eliminated
Feature #0 eliminated
Feature #14 eliminated
Feature #8 eliminated
Feature #21 eliminated
Step #2 out of 5

bestTest = 0.04307168082
bestIteration = 71

Shrink model to first 72 iterations.
Feature #10 eliminated
Feature #17 eliminated
Feature #19 eliminated
Step #3 out of 5

bestTest = 0.04309668493
bestIteration = 37

Shrink model to first 38 iterations.
Feature #9 eliminated
Feature #13 eliminated
Feature #16 eliminated
Step #4 out of 5

bestTest = 0.0398663503
bestIteration = 51

Shrink model to first 52 iterations.
Feature #18 eliminated
Feature #24 eliminated
Feature #22 eliminated
Step #5 out of 5

bestTest = 0.03897128664
bestIteration = 82

Shrink model to first 83 iterations.
Feature #6 eliminated
Feature #2 eliminated


In [9]:
import Utils as utils



print(rfe_dict['selected_features_names'])
utils.eval_models(dfX[rfe_dict['selected_features_names']], y, kf) 

['Laki', 'Umur', 'Umur Squared', 'APM SD', 'APM SMA', 'pendidikan_SMP', 'SD', 'Pengeluaran Bulanan', 'DPT', 'Weighted Sentiment']


,Method,RSME,MAE,R2
0,Linear Regression,0.040812,0.032367,0.292136
1,SVR,0.047896,0.038916,0.025071
2,Decision Tree,0.054813,0.044355,-0.276866
3,Random Forest,0.039495,0.032634,0.337066
4,XGBoost,0.043445,0.035171,0.197859
5,Catboost,0.038451,0.031515,0.371669
6,LightGBM,0.041618,0.033593,0.263880


### Hasil Evaluasi CatBoost RFE 

| No | Method            | RSME          | MAE           | R2           | Features                                                                                                                                                                         |
|----|-------------------|---------------|---------------|--------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  5 | Catboost          | 0,03616670817 | 0,02940776189 | 0,4440991577 | Perempuan, Umur Squared, APM SMA, pendidikan_SMP, SD, Pengeluaran Bulanan, Indeks Pembangunan Manusia, DPT, Weighted Sentiment                                                   |
|  5 | Catboost          | 0,03744954854 | 0,03101037007 | 0,4039639315 | Laki, Perempuan, Umur, Umur Squared, APM SMP, APM SMA, pendidikan_SMP, pendidikan_Universitas, SD, SMP, Pengeluaran Bulanan, Indeks Pembangunan Manusia, DPT, Weighted Sentiment |
|  0 | Linear Regression | 0,03747535333 | 0,03069829856 | 0,4031422453 | Laki, Perempuan, Umur, Umur Squared, APM SMP, APM SMA, pendidikan_SMP, pendidikan_Universitas, SD, SMP, Pengeluaran Bulanan, Indeks Pembangunan Manusia, DPT, Weighted Sentiment |
|  3 | Random Forest     | 0,03782791378 | 0,03047475347 | 0,3918591886 | Perempuan, Umur Squared, APM SMA, pendidikan_SMP, SD, Pengeluaran Bulanan, Indeks Pembangunan Manusia, DPT, Weighted Sentiment                                                   |
|  0 | Linear Regression | 0,03804527728 | 0,03089460299 | 0,3848502164 | Perempuan, Umur Squared, APM SMA, pendidikan_SMP, SD, Pengeluaran Bulanan, Indeks Pembangunan Manusia, DPT, Weighted Sentiment                                                   |


## Pendekatan Dimensionality Reduction

### Rough Set Reduction

Rough set reduction merupakan teknik seleksi fitur yang memanfaatkan teori rough set. Teknik ini mengelompokkan fitur berdasarkan nilai-nilai yang dimilikinya. Fitur yang memiliki nilai yang sama akan dikelompokkan dalam satu kelas. Teknik ini menghasilkan himpunan fitur yang disebut reduct.

Karena rough set tidak bisa digunakan pada data numeric, perlu dilakukan proses diskritisasi terlebih dahulu. Proses diskritisasi dilakukan dengan menggunakan algoritma D.local.discernibility.heuristic.RST. Algoritma ini menghasilkan cut values yang digunakan untuk membagi data menjadi beberapa interval. Setelah data diskritisasi, proses rough set reduction dapat dilakukan.
```r
library(RoughSets)
decision.table <- SF.asDecisionTable(dataset = data, decision.attr = 26)
cut.values <-D.local.discernibility.heuristic.RST(decision.table)
data.discretized <- SF.applyDecTable(decision.table, cut.values)
dim(data.discretized)
lapply(data.discretized, unique)
```

Data yang telah terdiskritisasi dapat digunakan untuk melakukan rough set reduction.

```r
library(Rcpp)

# Mengubah data diskrit menjadi numeric
data.PSO.100n <- data[c('Laju.Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'Umur.Squared', 'APM.SMP', 'APM.SMA', 'pendidikan_SD', 'pendidikan_SMP', 'Universitas', 'Pengeluaran.Bulanan', 'Indeks.Pembangunan.Manusia', 'DPT', 'Weighted.Sentiment','Partisipasi')]
table.pso.100 <- SF.asDecisionTable(dataset = data.PSO.100n, decision.attr = 15)
cut.equal.pso.100.4 <- D.discretize.equal.intervals.RST(table.pso.100, nOfIntervals = 6)
discretized.pso.100.4.equal <- SF.applyDecTable(table.pso.100, cut.equal.pso.100.4)
dim(discretized.pso.100.4.equal)
lapply(discretized.pso.100.4.equal, unique)

# Mengubah data diskrit menjadi numeric
reduct.equal.pso.100.4 <- FS.reduct.computation(discretized.pso.100.4.equal, method = "greedy.heuristic")
result.equal.pso.100.4 <- SF.applyDecTable(discretized.pso.100.4.equal, reduct.equal.pso.100.4)
print(colnames(result.equal.pso.100.4))
```

Karena rough set merupakan dimensionality reduction, maka rough set akan dibandingkan berdasarkan set awal yang digunakan. Set awal yang digunakan merupakan hasil PSO dari feature selection diatas

### Hasil Evaluasi Rough Set Reduction

PSO vs Rough Set Reduction (4 Interval)
| No | Method            | RSME    | MAE     | R2       | Method2 | RSME_RED | MAE_RED | R2_RED   |
|----|-------------------|---------|---------|----------|---------|----------|---------|----------|
| 0  | Linear Regression | 0,03657 | 0,03030 | 0,43162  | Reduced | 0,03742  | 0,03067 | 0,40501  |
| 1  | SVR               | 0,04803 | 0,03928 | 0,01972  | Reduced | 0,04754  | 0,03867 | 0,03936  |
| 2  | Decision Tree     | 0,05109 | 0,04087 | -0,10925 | Reduced | 0,05160  | 0,04125 | -0,13154 |
| 3  | Random   Forest   | 0,03829 | 0,03116 | 0,37694  | Reduced | 0,03870  | 0,03129 | 0,36347  |
| 4  | XGBoost           | 0,04155 | 0,03468 | 0,26620  | Reduced | 0,04257  | 0,03542 | 0,22990  |
| 5  | Catboost          | 0,03561 | 0,02932 | 0,46101  | Reduced | 0,03653  | 0,03031 | 0,43294  |
| 6  | LightGBM          | 0,03911 | 0,03088 | 0,35006  | Reduced | 0,03918  | 0,03096 | 0,34759  |

PSO vs Rough Set Reduction (6 Interval)
| No | Method            | RSME    | MAE     | R2       | Method2 | RSME_RED | MAE_RED | R2_RED   |
|----|-------------------|---------|---------|----------|---------|----------|---------|----------|
| 0  | Linear Regression | 0,03657 | 0,03030 | 0,43162  | Reduced | 0,03998  | 0,03264 | 0,32075  |
| 1  | SVR               | 0,04803 | 0,03928 | 0,01972  | Reduced | 0,04756  | 0,03867 | 0,03884  |
| 2  | Decision Tree     | 0,05280 | 0,04216 | -0,18460 | Reduced | 0,05075  | 0,04107 | -0,09441 |
| 3  | Random   Forest   | 0,03829 | 0,03116 | 0,37694  | Reduced | 0,03886  | 0,03163 | 0,35833  |
| 4  | XGBoost           | 0,04155 | 0,03468 | 0,26620  | Reduced | 0,04205  | 0,03518 | 0,24858  |
| 5  | Catboost          | 0,03561 | 0,02932 | 0,46101  | Reduced | 0,03621  | 0,02998 | 0,44282  |
| 6  | LightGBM          | 0,03911 | 0,03088 | 0,35006  | Reduced | 0,03915  | 0,03084 | 0,34871  |

GWO vs Rough Set Reduction (Greedy Heuristic Interval)

| No | Method            | RSME    | MAE     | R2       | Method2 | RSME_RED | MAE_RED | R2_RED   |
|----|-------------------|---------|---------|----------|---------|----------|---------|----------|
| 0  | Linear Regression | 0,03657 | 0,03030 | 0,43162  | Reduced | 0,03998  | 0,03264 | 0,32075  |
| 1  | SVR               | 0,04803 | 0,03928 | 0,01972  | Reduced | 0,04756  | 0,03867 | 0,03884  |
| 2  | Decision Tree     | 0,05280 | 0,04216 | -0,18460 | Reduced | 0,05075  | 0,04107 | -0,09441 |
| 3  | Random   Forest   | 0,03829 | 0,03116 | 0,37694  | Reduced | 0,03886  | 0,03163 | 0,35833  |
| 4  | XGBoost           | 0,04155 | 0,03468 | 0,26620  | Reduced | 0,04205  | 0,03518 | 0,24858  |
| 5  | Catboost          | 0,03561 | 0,02932 | 0,46101  | Reduced | 0,03621  | 0,02998 | 0,44282  |
| 6  | LightGBM          | 0,03911 | 0,03088 | 0,35006  | Reduced | 0,03915  | 0,03084 | 0,34871  |

### DIfference between Rough Set Reduction and PSO

4 Intervals
| RSME Diff | MAE Diff |  R2 Diff |
|:---------:|:--------:|:--------:|
| -0,00085  | -0,00037 | 0,02661  |
| 0,00048   | 0,00060  | -0,01964 |
| -0,00051  | -0,00038 | 0,02228  |
| -0,00041  | -0,00013 | 0,01347  |
| -0,00102  | -0,00075 | 0,03630  |
| -0,00092  | -0,00098 | 0,02807  |
| -0,00007  | -0,00008 | 0,00247  |

6 Intervals
| RSME Diff | MAE Diff |  R2 Diff |
|:---------:|:--------:|:--------:|
| -0,00341  | -0,00235 | 0,11087  |
| 0,00047   | 0,00061  | -0,01912 |
| 0,00205   | 0,00109  | -0,09019 |
| -0,00057  | -0,00047 | 0,01861  |
| -0,00050  | -0,00050 | 0,01762  |
| -0,00060  | -0,00065 | 0,01819  |
| -0,00004  | 0,00004  | 0,00135  |

Greedy Heuristic
| RSME Diff | MAE Diff |  R2 Diff |
|:---------:|:--------:|:--------:|
| -0,00649  | -0,00402 | 0,23256  |
| -0,00018  | -0,00019 | 0,00740  |
| 0,00369   | 0,00280  | -0,15819 |
| -0,00140  | -0,00112 | 0,04814  |
| -0,00222  | -0,00282 | 0,08370  |
| -0,00093  | -0,00018 | 0,03076  |
| -0,00105  | -0,00115 | 0,03526  |

Pada ketiga skenario tersebut, rough set reduction banyak mengurangi akurasi model, kecuali pada decision tree. Pada model - model terbaik seperti CatBoost dan Linear Regression, penggunaan rough set reduction cenderun mengurangi performa model. Oleh karena itu, rough set reduction tidak digunakan pada skenario selanjutnya.